In [4]:

import os
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.layers import Dropout 
from keras.utils import np_utils 
from keras.layers import Flatten
from keras.layers import Dense 
from keras.constraints import maxnorm
from keras.optimizers import SGD 
import keras

Using TensorFlow backend.


In [1]:
pwd

'D:\\JupyterNotebook\\Assignment2'

# Loading dataset

In [2]:
def get_annotations_map():
    valAnnotationsPath = './tiny-imagenet-200/tiny-imagenet-200/val/val_annotations.txt'
    valAnnotationsFile = open(valAnnotationsPath, 'r')
    valAnnotationsContents = valAnnotationsFile.read()
    valAnnotations = {}

    for line in valAnnotationsContents.splitlines():
        pieces = line.strip().split()
        valAnnotations[pieces[0]] = pieces[1]

    return valAnnotations

In [6]:


def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,32,32],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,32,32],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    valPath=path + '/val/images'
    for sChild in os.listdir(valPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(valPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


    print('finished loading test images')#+str(i)

    return X_train,y_train,X_test,y_test

if __name__ == "__main__":
    import matplotlib.pyplot as plt

    path='./tiny-imagenet-200/tiny-imagenet-200'
    X_train,y_train,X_test,y_test=load_images(path,200)#input data path & numbers of classes
    

    fig1 = plt.figure()
    fig1.suptitle('Train data')
    ax1 = fig1.add_subplot(221)
    ax1.axis("off")
    ax1.imshow(np.transpose(X_train[0],(1,2,0)))
    print(y_train[0])
    ax2 = fig1.add_subplot(222)
    ax2.axis("off")
    ax2.imshow(np.transpose(X_train[499],(1,2,0)))
    print(y_train[499])
    ax3 = fig1.add_subplot(223)
    ax3.axis("off")
    ax3.imshow(np.transpose(X_train[500],(1,2,0)))
    print(y_train[500])
    ax4 = fig1.add_subplot(224)
    ax4.axis("off")
    ax4.imshow(np.transpose(X_train[999],(1,2,0)))
    print(y_train[999])

    plt.show()

    fig2 = plt.figure()
    fig2.suptitle('Test data')
    ax1 = fig2.add_subplot(221)
    ax1.axis("off")
    ax1.imshow(np.transpose(X_test[0],(1,2,0)))
    print(y_test[0])
    ax2 = fig2.add_subplot(222)
    ax2.axis("off")
    ax2.imshow(np.transpose(X_test[49],(1,2,0)))
    print(y_test[49])
    ax3 = fig2.add_subplot(223)
    ax3.axis("off")
    ax3.imshow(np.transpose(X_test[50],(1,2,0)))
    print(y_test[50])
    ax4 = fig2.add_subplot(224)
    ax4.axis("off")
    ax4.imshow(np.transpose(X_test[99],(1,2,0)))
    print(y_test[99])
    
    plt.show()

Loading 200 classes
loading training images...


ValueError: axes don't match array

In [4]:
X_train.shape

(100000, 3, 32, 32)

In [5]:
y_train.shape

(100000,)

# Model

In [6]:
# normalize inputs from 0-255 to 0.0-1.0 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train = X_train / 255.0 
X_test = X_test / 255.0

In [7]:
# one hot encode outputs 
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 
num_classes = y_test.shape[1]

In [26]:

# Create the model 
model = Sequential() 
model.add(Conv2D(32, (2, 2), input_shape=(3, 32, 32), activation='relu', padding='same')) 
model.add(Dropout(0.5)) 
model.add(Conv2D(64, (2, 2), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.5)) 



model.add(Flatten()) 
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu', kernel_constraint=maxnorm(3))) 
model.add(Dropout(0.5)) 
model.add(Dense(1024, activation='elu', kernel_constraint=maxnorm(3))) 
model.add(Dropout(0.5)) 
model.add(Dense(num_classes, activation='softmax'))

In [9]:

# Compile model e
epochs = 2
lrate = 0.01

In [27]:
decay = lrate/epochs 
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) 
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 3, 32, 32)         4128      
_________________________________________________________________
dropout_26 (Dropout)         (None, 3, 32, 32)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 3, 32, 64)         8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 1, 16, 64)         0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 1, 16, 64)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 1024)              0         
__________

In [28]:
# Fit the model 
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64) 
# Final evaluation of the model 
scores = model.evaluate(X_test, y_test, verbose=2) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 100000 samples, validate on 10000 samples
Epoch 1/2
100000/100000 [==============================] - 53s 526us/step - loss: 5.3000 - acc: 0.0049 - val_loss: 5.2980 - val_acc: 0.0056
Epoch 2/2
100000/100000 [==============================] - 51s 509us/step - loss: 5.2982 - acc: 0.0051 - val_loss: 5.2979 - val_acc: 0.0050
Accuracy: 0.50%


# Exp4

In [29]:
keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [30]:
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [31]:
model = InceptionResNetV2()

In [32]:
IMAGE_SIZE    = (32, 32)

net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

ValueError: Input size must be at least 75x75; got `input_shape=(32, 32, 3)`

In [33]:
import tensorflow as tf

In [34]:
tf.VERSION

'1.12.0'